In [1]:
# import sys
# sys.path.insert(0, '../Analysis')

import helpers as h
import empatica_helpers as eh
import inquisit_helpers as ih
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
reload(h), reload(eh), reload(ih)

# Global settings
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:.2f}'.format
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('seaborn-v0_8-notebook') # plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

# print(plt.style.available)

## Full preprocessing pipeline

In [14]:
reload(eh), reload(ih), reload(h)

sr = 32
wl = 24 # Window length in seconds

# Full pipeline
# e_raw, _ = eh.load_empatica(data_folder='input/empatica/', useIBI=False, save=True, plotTrimmings=False, desired_sampling_rate=sr)
# i_raw = ih.load_inquisit(data_folder='input/inquisit/', save=True)
# ei_raw = h.combine_empatica_and_inquisit(e_raw, i_raw, save=True, sr=sr)
# ei_prep = h.clean_scale_filter(save=True, normalise=True, sr=sr, window_length=wl)
# X, y, p = h.prepare_for_vae(sr=sr, wl=wl, filepath="output/ei_prep.csv", save=True)

X_train, X_val, X_test, y_train, y_val, y_test, p_train, p_val, p_test = h.prepare_train_val_test_sets(filenames=['output/dl_X_wl24_sr32.pkl', 'output/dl_y_wl24_sr32.pkl', 'output/dl_p_wl24_sr32.pkl'])

# Back to input for ML models
# X_train_aggregates = h.prepare_for_ml(X_train)
# X_val_aggregates = h.prepare_for_ml(X_val)
# X_test_aggregates = h.prepare_for_ml(X_test)

Train size:  80.23809523809524
Val size:  8.333333333333332
Test size:  11.428571428571429
Size: : (1011, 768, 6)


In [22]:
reload(h)

# Calculate the aggregates for the training set
X_train_aggregates = h.prepare_for_ml(X_train)

print(X_train_aggregates.shape)
print(X_train.shape)

(1011, 36)
(1011, 768, 6)


/Users/koenraijer/Documents/00_Werk_en_studie/Msc_Data_Science_&_Society/Thesis/Analysis/helpers.py:399: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  for train_index, test_index in gkf.split(X_train, y_train, groups):


**On VAE preprocessing**

The model expects data as a Numpy array with dimensions [samples, time points, variables]. My data is a multivariate timeseries with binary labels. It is sampled at 64 Hz. I think this means I should take labelled slices of my data that describe a multivariate timeseries at several timepoints. So if I'm looking to use an 8s sample that is labelled intrusion or not, I get [sample_index, 0:512, n_features] per sample.

## NON-CLASS IMPLEMENTATION: Conditional recurrent (LSTM) VAE

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='total_loss', mode="min", patience=4)

vae = LSTM_VAE(input_shape=X_train.shape, int_dim=64, latent_dim=12)
vae.compile(optimizer='adam')
history = vae.fit(x=X_train, y=y_train, epochs=100, batch_size=32, callbacks=[early_stopping])

In [ ]:
from tensorflow import keras, shape
import tensorflow.keras.backend as K
from tensorflow.keras.layers import(LSTM, Dense, Input, Lambda, RepeatVector, TimeDistributed, Concatenate)
from tensorflow.keras.losses import mse
from tensorflow.keras.models import Model

def create_lstm_vae_model(time_steps, number_of_features, int_dim, latent_dim, condition_dim):
    """
    Creates a conditional LSTM Variational Autoencoder (VAE) model.

    Parameters:
    - time_steps (int): The number of time steps in the input sequence.
    - number_of_features (int): The number of features in the input sequence.
    - int_dim (int): The dimensionality of the intermediate LSTM layer.
    - latent_dim (int): The dimensionality of the latent space.
    - condition_dim (int): The dimensionality of the condition.

    Returns:
    - encoder (Model): The encoder model.
    - decoder (Model): The decoder model.
    - lstm_vae (Model): The LSTM VAE model.

    """

    def vae_sampling(args):
        """
        Sampling function for the VAE.

        Parameters:
        - args (tuple): Tuple containing the mean and log variance of the latent space.

        Returns:
        - Sampled latent space vector.

        """
        z_mean, z_log_sigma = args
        batch_size = shape(z_mean)[0]
        latent_dim = shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0, stddev=1)
        return z_mean + K.exp(z_log_sigma / 2) * epsilon

    # Encoder
    input_x = Input(shape=(time_steps, number_of_features))
    condition_input = Input(shape=(condition_dim,))
    encoder_input = Concatenate()([input_x, RepeatVector(time_steps)(condition_input)])

    encoder_LSTM_int = LSTM(int_dim, return_sequences=True)(encoder_input)
    encoder_LSTM_latent = LSTM(latent_dim, return_sequences=False)(encoder_LSTM_int)

    z_mean = Dense(latent_dim)(encoder_LSTM_latent)
    z_log_sigma = Dense(latent_dim)(encoder_LSTM_latent)
    z_encoder_output = Lambda(vae_sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

    encoder = Model([input_x, condition_input], [z_mean, z_log_sigma, z_encoder_output])

    # Decoder
    decoder_input = Input(shape=(latent_dim,))
    decoder_condition_input = Input(shape=(condition_dim,))
    decoder_repeated = RepeatVector(time_steps)(Concatenate()([decoder_input, decoder_condition_input]))

    decoder_LSTM_int = LSTM(int_dim, return_sequences=True)(decoder_repeated)
    decoder_LSTM = LSTM(number_of_features, return_sequences=True)(decoder_LSTM_int)
    decoder_output = TimeDistributed(Dense(number_of_features))(decoder_LSTM)
    decoder = Model([decoder_input, decoder_condition_input], decoder_output)

    # VAE
    output = decoder([encoder([input_x, condition_input])[2], condition_input])
    lstm_vae = keras.Model([input_x, condition_input], output, name='lstm_vae')

    # Loss
    rec_loss = K.mean(mse(input_x, output)) * number_of_features
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
    vae_loss = rec_loss + kl_loss

    lstm_vae.add_loss(vae_loss)

    return encoder, decoder, lstm_vae

def generate_lstm_vae_samples(decoder, n, latent_dim, condition):
    '''
        Generate random samples from the LSTM VAE.

        n : int
            The number of samples to generate.

        Returns
        -------
        A numpy array of shape (n, time_steps, number_of_features) containing the generated samples.
    '''
    # Sample from the standard normal distribution
    z_samples = np.random.normal(size=(n, latent_dim))

    # Decode the samples
    gen = decoder.predict([z_samples, np.repeat(condition, n, axis=0)])

    return gen

### Generating new data from the VAE's latent space

In [ ]:
# Define the number of samples to generate for each class
n_samples = 500

# Define the conditions for each class
condition_0 = np.array([[0]])
condition_1 = np.array([[1]])

# Generate samples for each class
samples_0 = generate_lstm_vae_samples(decoder, n_samples, latent_dim, condition_0)
samples_1 = generate_lstm_vae_samples(decoder, n_samples, latent_dim, condition_1)

## Defining a non-conditional LSTM-VAE

In [100]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow import shape
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model

# Source: https://github.com/meliksahturker/LSTM-VAE/tree/main?tab=readme-ov-file 

class LSTM_VAE(keras.Model):
    def __init__(self, input_shape, int_dim, latent_dim, **kwargs):
        super().__init__(**kwargs)
        self.int_dim = int_dim
        self.latent_dim = latent_dim
        self.num_features = input_shape[2]
        self.time_steps = input_shape[1]
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    def build_encoder(self):
        """
        Builds the encoder part of the VAE.
        """

        def Sampling(args):
            """
            Sampling function for the VAE.
            Parameters: args (tuple): Tuple containing the mean and log variance of the latent space.
            Returns: Sampled latent space vector.
            """
            z_mean, z_log_var = args
            batch_size = shape(z_mean)[0] # Number of samples in the batch
            latent_dim = shape(z_mean)[1] # Dimensionality of the latent space
            epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0, stddev=1) # epsilon = irreducible error. Generates random noise to add to our reparameterisation trick value
            # Reparameterisation trick
            # - `K.exp()` takes the exponential of our log variance to obtain the variance. 
            # - `z_log_var / 2` is equivalent to taking the square root of the variance (standard deviation)
            # - `* epsilon` to get a random value from a normal distribution with mean 0 and standard deviation 1
            return z_mean + K.exp(z_log_var / 2) * epsilon

        # --------------------- ENCODER ---------------------
        x = Input(shape=(self.time_steps, self.num_features))  # Keras adds None to the shape for the batch size: (None, time_steps, number_of_features)
        encoder_LSTM_intermediate = LSTM(self.int_dim, return_sequences=True, name="encoder_LSTM_intermediate")(x)
        encoder_LSTM_latent = LSTM(self.latent_dim, return_sequences=False, name="encoder_LSTM_latent")(encoder_LSTM_intermediate)

        # These layers' outputs will be trained to represent the mean and log variance of the latent space
        z_mean = Dense(self.latent_dim, name="z_mean")(encoder_LSTM_latent) # Mean(s) of the latent space
        z_log_var = Dense(self.latent_dim, name="z_log_var")(encoder_LSTM_latent) # Log variance(s) of the latent space. Log is used to ensure so its exponent (which we'll calculate later) is always positive.

        # A Lambda layer is used to sample from the latent space by passing the mean and log variance to the vae_sampling function
        z = Lambda(Sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])

        encoder = Model(inputs=x, outputs=[z_mean, z_log_var, z], name="encoder") # z_mean and z_log_sigma are returned for loss calculation, z_encoder_output is the output of the encoder and will be used as input to the decoder

        # SIDENOTE: Keras is able to trace back the computation graph from the output of the encoder to the input, so it will infer the structure of the encoder from its output and input layers.

        return encoder

    def build_decoder(self):
        """
        Builds the decoder part of the VAE.
        """
        decoder_input = Input(shape=(self.latent_dim,)) # Input to the decoder is the latent space vector z
        decoder_repeated = RepeatVector(self.time_steps)(decoder_input) # Repeats the latent space vector (z) for the number of time steps, to match the input shape of the LSTM.
        decoder_LSTM_intermediate = LSTM(self.int_dim, return_sequences=True)(decoder_repeated) # Transforms (batch_size, time_steps, latent_dim) to (batch_size, time_steps, int_dim). 
        decoder_LSTM = LSTM(self.num_features, return_sequences=True)(decoder_LSTM_intermediate) # Transforms (batch_size, time_steps, int_dim) to (batch_size, time_steps, number_of_features). 
        decoder_output = TimeDistributed(Dense(self.num_features))(decoder_LSTM) # Contains Dense layer at the end to be able to produce high absolute values, since LSTM activations are tanh.
        decoder = Model(inputs=decoder_input, outputs=decoder_output, name="decoder") 

        return decoder

    @property
    def metrics(self):
            """
            Returns a list of metrics used for tracking during training.
            Returns:list: A list of metrics including total loss, reconstruction loss, and KL divergence loss.
            """
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

    def train_step(self, data):
        """
        Performs a single training step on the given data.
        Args: data: The input data for training.
        Returns: A dictionary containing the following metrics: loss; The total loss value, reconstruction_loss; The reconstruction loss value, kl_loss; The KL divergence loss value.
        """
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction),
                    axis=(0, 1),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        """
        Performs the forward pass of the model.
        Args: inputs: The input data.
        Returns: The reconstructed output.
        """
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return reconstruction

    def test_step(self, data):
        """
        Performs a single testing step of the model.
        Args: data: The input data for testing.
        Returns: A dictionary containing the loss values for the total loss, reconstruction loss, and KL divergence loss.
        """
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.mse(data, reconstruction),
                axis=(0, 1),
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = reconstruction_loss + kl_loss
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def generate_samples(self, n):
        '''
            Generate random samples from the LSTM VAE.

            n : int
                The number of samples to generate.

            Returns
            -------
            A numpy array of shape (n, time_steps, number_of_features) containing the generated samples.
        '''
        import numpy as np

        # Sample from the standard normal distribution
        z_samples = np.random.normal(size=(n, self.latent_dim))

        # Decode the samples
        gen = self.decoder.predict(z_samples)

        return gen

## Defining a conditional LSTM-VAE

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate, Flatten
from tensorflow.keras import backend as K
from tensorflow import shape
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow_addons.rnn import PeepholeLSTMCell

# Source: https://github.com/meliksahturker/LSTM-VAE/tree/main?tab=readme-ov-file 

class LSTM_VAE(Model):
    def __init__(self, input_shape, int_dim, latent_dim, condition_dim=1, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.int_dim = int_dim
        self.latent_dim = latent_dim
        self.num_features = input_shape[2]
        self.time_steps = input_shape[1]
        self.condition_dim = condition_dim
        self.condition_input = Input(shape=(condition_dim,)) # Shape of the condition input

        # Everything needed for the encoder and decoder should be defined before calling the encoder and decoder functions
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)
            tf.random.set_seed(self.seed)
    
    def build_encoder(self):
        """
        Builds the encoder part of the VAE.
        """

        def Sampling(args):
            """
            Sampling function for the VAE.
            Parameters: args (tuple): Tuple containing the mean and log variance of the latent space.
            Returns: Sampled latent space vector.
            """
            z_mean, z_log_var = args
            batch_size = shape(z_mean)[0] # Number of samples in the batch
            latent_dim = shape(z_mean)[1] # Dimensionality of the latent space
            epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0, stddev=1) # epsilon = irreducible error. Generates random noise to add to our reparameterisation trick value
            # Reparameterisation trick
            # - `K.exp()` takes the exponential of our log variance to obtain the variance. 
            # - `z_log_var / 2` is equivalent to taking the square root of the variance (standard deviation)
            # - `* epsilon` to get a random value from a normal distribution with mean 0 and standard deviation 1
            return z_mean + K.exp(z_log_var / 2) * epsilon

        # --------------------- ENCODER ---------------------
        # Creating the input layer
        x = Input(shape=(self.time_steps, self.num_features))  # Keras adds None to the shape for the batch size: (None, time_steps, number_of_features)
        encoder_input = Concatenate()([x, RepeatVector(self.time_steps)(self.condition_input)])

        # Creating the LSTM layers
        # PeepholeLSTMCell(n_hidden[0], activation=tf.nn.tanh, name="encoder") # Docs: https://www.tensorflow.org/addons/api_docs/python/tfa/rnn/PeepholeLSTMCell
        encoder_LSTM_intermediate = LSTM(self.int_dim, return_sequences=True, name="encoder_LSTM_intermediate")(encoder_input)
        encoder_LSTM_latent = LSTM(self.latent_dim, return_sequences=False, name="encoder_LSTM_latent")(encoder_LSTM_intermediate)

        # These layers' outputs will be trained to represent the mean and log variance of the latent space
        z_mean = Dense(self.latent_dim, name="z_mean")(encoder_LSTM_latent) # Mean(s) of the latent space
        z_log_var = Dense(self.latent_dim, name="z_log_var")(encoder_LSTM_latent) # Log variance(s) of the latent space. Log is used to ensure so its exponent (which we'll calculate later) is always positive.

        # A Lambda layer is used to sample from the latent space by passing the mean and log variance to the vae_sampling function
        z = Lambda(Sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])

        encoder = Model(inputs=[x, self.condition_input], outputs=[z_mean, z_log_var, z], name="encoder") # z_mean and z_log_sigma are returned for loss calculation, z_encoder_output is the output of the encoder and will be used as input to the decoder

        # SIDENOTE: Keras is able to trace back the computation graph from the output of the encoder to the input, so it will infer the structure of the encoder from its output and input layers.

        return encoder

    def build_decoder(self):
        """
        Builds the decoder part of the VAE.
        """
        decoder_input = Input(shape=(self.latent_dim,)) # Input to the decoder is the latent space vector z
        condition_input_repeated = Flatten()(RepeatVector(self.latent_dim)(self.condition_input))
        decoder_input_concat = Concatenate()([decoder_input, condition_input_repeated])
        decoder_repeated = RepeatVector(self.time_steps)(decoder_input_concat) # Repeats the latent space vector (z) for the number of time steps, to match the input shape of the LSTM.
        decoder_LSTM_intermediate = LSTM(self.int_dim, return_sequences=True)(decoder_repeated) # Transforms (batch_size, time_steps, latent_dim) to (batch_size, time_steps, int_dim). 
        decoder_LSTM = LSTM(self.num_features, return_sequences=True)(decoder_LSTM_intermediate) # Transforms (batch_size, time_steps, int_dim) to (batch_size, time_steps, number_of_features). 
        decoder_output = TimeDistributed(Dense(self.num_features))(decoder_LSTM) # Contains Dense layer at the end to be able to produce high absolute values, since LSTM activations are tanh.
        decoder = Model(inputs=[decoder_input, self.condition_input], outputs=decoder_output, name="decoder") 
        
        return decoder

    @property
    def metrics(self):
            """
            Returns a list of metrics used for tracking during training.
            Returns:list: A list of metrics including total loss, reconstruction loss, and KL divergence loss.
            """
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

    def call(self, inputs):
        """
        Performs the forward pass of the model.
        Args: inputs: The input data.
        Returns: The reconstructed output.
        """
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return reconstruction

    def calculate_losses(self, x, condition):
        """
        Calculates the total loss, reconstruction loss, and KL divergence loss for a given input and condition.

        Parameters:
            x (tensor): The input tensor.
            condition (tensor): The condition tensor.

        Returns:
            total_loss (tensor): The total loss.
            reconstruction_loss (tensor): The reconstruction loss.
            kl_loss (tensor): The KL divergence loss.
        """
        z_mean, z_log_var, z = self.encoder([x, condition])
        reconstruction = self.decoder([z, condition])
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.mse(x, reconstruction),
                axis=(0, 1),
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = reconstruction_loss + kl_loss
        return total_loss, reconstruction_loss, kl_loss

    @tf.function
    def train_step(self, data):
        """
        Performs a single training step on the given data.

        Args:
            data: A tuple containing the input data `x` and the condition `condition`.

        Returns:
            A dictionary containing the following metrics:
            - "loss": The total loss value.
            - "reconstruction_loss": The reconstruction loss value.
            - "kl_loss": The KL divergence loss value.
        """
        x, condition = data
        with tf.GradientTape() as tape:
            total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    @tf.function
    def test_step(self, data):
        """
        Perform a single testing step of the model.

        Args:
            data (tuple): A tuple containing the input data `x` and the condition `condition`.

        Returns:
            dict: A dictionary containing the calculated losses for the testing step, including the total loss, reconstruction loss, and KL loss.
        """
        x, condition = data
        total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def generate_samples(self, n, condition):
        '''
            Generate random samples from the LSTM VAE.

            n : int : The number of samples to generate.
            condition : numpy array : The condition to generate the samples for.

            Returns
            -------
            A numpy array of shape (n, time_steps, number_of_features) containing the generated samples.
        '''

        # Sample from the standard normal distribution
        z_samples = np.random.normal(size=(n, self.latent_dim))

        # Decode the samples
        gen = self.decoder.predict([z_samples, np.repeat(condition, n, axis=0)])

        return gen

/Users/koenraijer/Documents/00_Werk_en_studie/Msc_Data_Science_&_Society/Thesis/Analysis/.conda/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/koenraijer/Documents/00_Werk_en_studie/Msc_Data_Science_&_Society/Thesis/Analysis/.conda/lib/python3.11/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently usi

ModuleNotFoundError: No module named 'keras.src.engine'

## Hyperparameter optimisation

In [21]:
from hyperopt import hp
from keras.optimizers import Adam, RMSprop, SGD, Adadelta
from keras.regularizers import l1_l2
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

# Define the hyperparameter space
space = {
    'int_dim': hp.quniform('int_dim', 32, 256, 1),
    'latent_dim': hp.quniform('latent_dim', 2, 20, 1),
    'batch_size': 32,
}

def train_model(params):
    # Create the model
    vae = LSTM_VAE(input_shape=X_train.shape, int_dim=int(params['int_dim']), latent_dim=int(params['latent_dim']))
    vae.compile(optimizer='adam')
    early_stopping = EarlyStopping(monitor='total_loss', mode="min", patience=4)
    history = vae.fit(x=X_train, y=y_train, batch_size=int(params['batch_size']), callbacks=[early_stopping], epochs=100)
    return history.history['total_loss'][-1]

In [ ]:
from hyperopt import fmin, tpe, Trials

# Objective function
def objective(params):
    return train_model(params)

best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=Trials())

print("Best parameters found: ", best_params)

## Added `seed` and `PeepholeLSTM`

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate, Flatten, RNN
from tensorflow.keras import backend as K
from tensorflow import shape
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow_addons.rnn import PeepholeLSTMCell
from tensorflow.keras.initializers import GlorotUniform
import os
import joblib

# Source: https://github.com/meliksahturker/LSTM-VAE/tree/main?tab=readme-ov-file 

class LSTM_VAE(Model):
    def __init__(self, input_shape, int_dim, latent_dim, condition_dim=1, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.int_dim = int_dim
        self.latent_dim = latent_dim
        self.num_features = input_shape[2]
        self.time_steps = input_shape[1]
        self.condition_dim = condition_dim
        self.condition_input = Input(shape=(condition_dim,)) # Shape of the condition input

        self.seed = seed
        self.layer_seed = 0 
        if self.seed is not None:
            np.random.seed(self.seed)
            tf.random.set_seed(self.seed)
            
        # Everything needed for the encoder and decoder should be defined before calling the encoder and decoder functions
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
    
    def build_encoder(self):
        """
        Builds the encoder part of the VAE.
        """

        def Sampling(args):
            """
            Sampling function for the VAE.
            Parameters: args (tuple): Tuple containing the mean and log variance of the latent space.
            Returns: Sampled latent space vector.
            """
            z_mean, z_log_var = args
            batch_size = shape(z_mean)[0] # Number of samples in the batch
            latent_dim = shape(z_mean)[1] # Dimensionality of the latent space
            epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0, stddev=1) # epsilon = irreducible error. Generates random noise to add to our reparameterisation trick value
            # Reparameterisation trick
            # - `K.exp()` takes the exponential of our log variance to obtain the variance. 
            # - `z_log_var / 2` is equivalent to taking the square root of the variance (standard deviation)
            # - `* epsilon` to get a random value from a normal distribution with mean 0 and standard deviation 1
            return z_mean + K.exp(z_log_var / 2) * epsilon

        # --------------------- ENCODER ---------------------
        # Creating the input layer
        x = Input(shape=(self.time_steps, self.num_features))  # Keras adds None to the shape for the batch size: (None, time_steps, number_of_features)
        encoder_input = Concatenate()([x, RepeatVector(self.time_steps)(self.condition_input)])

        # Creating the LSTM layers
        encoder_LSTM_intermediate = RNN(PeepholeLSTMCell(self.int_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True, name="encoder_LSTM_intermediate")(encoder_input)
        self.layer_seed += 1
        encoder_LSTM_latent = RNN(PeepholeLSTMCell(self.latent_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=False, name="encoder_LSTM_latent")(encoder_LSTM_intermediate)
        self.layer_seed += 1

        # NOTE: GlorotUniform is equivalent to Xavier activation function (recommended with tanh activation functions, the default in LSTMs).

        # These layers' outputs will be trained to represent the mean and log variance of the latent space
        z_mean = Dense(self.latent_dim, name="z_mean")(encoder_LSTM_latent) # Mean(s) of the latent space
        z_log_var = Dense(self.latent_dim, name="z_log_var")(encoder_LSTM_latent) # Log variance(s) of the latent space. Log is used to ensure so its exponent (which we'll calculate later) is always positive.

        # A Lambda layer is used to sample from the latent space by passing the mean and log variance to the vae_sampling function
        z = Lambda(Sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])

        encoder = Model(inputs=[x, self.condition_input], outputs=[z_mean, z_log_var, z], name="encoder") # z_mean and z_log_sigma are returned for loss calculation, z_encoder_output is the output of the encoder and will be used as input to the decoder

        # SIDENOTE: Keras is able to trace back the computation graph from the output of the encoder to the input, so it will infer the structure of the encoder from its output and input layers.

        return encoder

    def build_decoder(self):
        """
        Builds the decoder part of the VAE.
        """
        decoder_input = Input(shape=(self.latent_dim,)) # Input to the decoder is the latent space vector z
        condition_input_repeated = Flatten()(RepeatVector(self.latent_dim)(self.condition_input))
        decoder_input_concat = Concatenate()([decoder_input, condition_input_repeated])
        decoder_repeated = RepeatVector(self.time_steps)(decoder_input_concat) # Repeats the latent space vector (z) for the number of time steps, to match the input shape of the LSTM.
        decoder_LSTM_intermediate = RNN(PeepholeLSTMCell(self.int_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True)(decoder_repeated) # Transforms (batch_size, time_steps, latent_dim) to (batch_size, time_steps, int_dim). 
        self.layer_seed += 1
        decoder_LSTM = RNN(PeepholeLSTMCell(self.num_features, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True)(decoder_LSTM_intermediate)
        decoder_output = TimeDistributed(Dense(self.num_features))(decoder_LSTM) # Contains Dense layer at the end to be able to produce high absolute values, since LSTM activations are tanh.
        decoder = Model(inputs=[decoder_input, self.condition_input], outputs=decoder_output, name="decoder") 
        
        return decoder

    @property
    def metrics(self):
            """
            Returns a list of metrics used for tracking during training.
            Returns:list: A list of metrics including total loss, reconstruction loss, and KL divergence loss.
            """
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

    def call(self, inputs):
        """
        Performs the forward pass of the model.
        Args: inputs: The input data.
        Returns: The reconstructed output.
        """
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return reconstruction

    def calculate_losses(self, x, condition):
        """
        Calculates the total loss, reconstruction loss, and KL divergence loss for a given input and condition.

        Parameters:
            x (tensor): The input tensor.
            condition (tensor): The condition tensor.

        Returns:
            total_loss (tensor): The total loss.
            reconstruction_loss (tensor): The reconstruction loss.
            kl_loss (tensor): The KL divergence loss.
        """
        z_mean, z_log_var, z = self.encoder([x, condition])
        reconstruction = self.decoder([z, condition])
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.mse(x, reconstruction),
                axis=(0, 1),
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = reconstruction_loss + kl_loss
        return total_loss, reconstruction_loss, kl_loss

    @tf.function
    def train_step(self, data):
        """
        Performs a single training step on the given data.

        Args:
            data: A tuple containing the input data `x` and the condition `condition`.

        Returns:
            A dictionary containing the following metrics:
            - "loss": The total loss value.
            - "reconstruction_loss": The reconstruction loss value.
            - "kl_loss": The KL divergence loss value.
        """
        x, condition = data
        with tf.GradientTape() as tape:
            total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    @tf.function
    def test_step(self, data):
        """
        Perform a single testing step of the model.

        Args:
            data (tuple): A tuple containing the input data `x` and the condition `condition`.

        Returns:
            dict: A dictionary containing the calculated losses for the testing step, including the total loss, reconstruction loss, and KL loss.
        """
        x, condition = data
        total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def save_weights(self, model_dir, file_pref):
        encoder_wts = self.encoder.get_weights()
        decoder_wts = self.decoder.get_weights()
        joblib.dump(encoder_wts, os.path.join(model_dir, f"{file_pref}encoder_wts.h5"))
        joblib.dump(decoder_wts, os.path.join(model_dir, f"{file_pref}decoder_wts.h5"))

    def load_weights(self, model_dir, file_pref):
        encoder_wts = joblib.load(os.path.join(model_dir, f"{file_pref}encoder_wts.h5"))
        decoder_wts = joblib.load(os.path.join(model_dir, f"{file_pref}decoder_wts.h5"))
        self.encoder.set_weights(encoder_wts)
        self.decoder.set_weights(decoder_wts)

    def save(self, model_dir, file_pref):
        self.save_weights(model_dir, file_pref)
        dict_params = {
            "int_dim": self.int_dim,
            "latent_dim": self.latent_dim,
            "num_features": self.num_features,
            "time_steps": self.time_steps,
            "condition_dim": self.condition_dim,
        }
        params_file = os.path.join(model_dir, f"{file_pref}parameters.pkl")
        joblib.dump(dict_params, params_file)
        
    def generate_samples(self, n, condition):
        '''
            Generate random samples from the LSTM VAE.

            n : int : The number of samples to generate.
            condition : numpy array : The condition to generate the samples for.

            Returns
            -------
            A numpy array of shape (n, time_steps, number_of_features) containing the generated samples.
        '''

        # Sample from the standard normal distribution
        z_samples = np.random.normal(size=(n, self.latent_dim))

        # Decode the samples
        gen = self.decoder.predict([z_samples, np.repeat(condition, n, axis=0)])

        return gen

## Most recent

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate, Flatten, RNN
from tensorflow.keras import backend as K
from tensorflow import shape
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow_addons.rnn import PeepholeLSTMCell
from tensorflow.keras.initializers import GlorotUniform
import os
import joblib

# Source: https://github.com/meliksahturker/LSTM-VAE/tree/main?tab=readme-ov-file 

class LSTM_VAE(Model):
    def __init__(self, input_shape, int_dim=36, latent_dim=18, condition_dim=1, batch_size=32, learning_rate=0.001, reconstruction_wt = 1.0, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.int_dim = int_dim
        self.latent_dim = latent_dim
        self.num_features = input_shape[2]
        self.time_steps = input_shape[1]
        self.condition_dim = condition_dim
        self.condition_input = Input(shape=(condition_dim,)) # Shape of the condition input
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.reconstruction_wt = reconstruction_wt
        self.seed = seed
        self.layer_seed = 0 
        if self.seed is not None:
            np.random.seed(self.seed)
            tf.random.set_seed(self.seed)
            
        # Everything needed for the encoder and decoder should be defined before calling the encoder and decoder functions
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
    
    def build_encoder(self):
        """
        Builds the encoder part of the VAE.
        """

        def Sampling(args):
            """
            Sampling function for the VAE.
            Parameters: args (tuple): Tuple containing the mean and log variance of the latent space.
            Returns: Sampled latent space vector.
            """
            z_mean, z_log_var = args
            batch_size = shape(z_mean)[0] # Number of samples in the batch
            latent_dim = shape(z_mean)[1] # Dimensionality of the latent space
            epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0, stddev=1) # epsilon = irreducible error. Generates random noise to add to our reparameterisation trick value
            # Reparameterisation trick
            # - `K.exp()` takes the exponential of our log variance to obtain the variance. 
            # - `z_log_var / 2` is equivalent to taking the square root of the variance (standard deviation)
            # - `* epsilon` to get a random value from a normal distribution with mean 0 and standard deviation 1
            return z_mean + K.exp(z_log_var / 2) * epsilon

        # --------------------- ENCODER ---------------------
        # Creating the input layer
        x = Input(shape=(self.time_steps, self.num_features))  # Keras adds None to the shape for the batch size: (None, time_steps, number_of_features)
        encoder_input = Concatenate()([x, RepeatVector(self.time_steps)(self.condition_input)]) # Repeat condition for each timsestep. 

        # Creating the LSTM layers
        encoder_LSTM_intermediate = RNN(PeepholeLSTMCell(self.int_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True, name="encoder_LSTM_intermediate")(encoder_input)
        self.layer_seed += 1
        encoder_LSTM_latent = RNN(PeepholeLSTMCell(self.latent_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=False, name="encoder_LSTM_latent")(encoder_LSTM_intermediate)
        self.layer_seed += 1

        # NOTE: GlorotUniform is equivalent to Xavier activation function (recommended with tanh activation functions, the default in LSTMs).

        # These layers' outputs will be trained to represent the mean and log variance of the latent space
        z_mean = Dense(self.latent_dim, name="z_mean")(encoder_LSTM_latent) # Mean(s) of the latent space
        z_log_var = Dense(self.latent_dim, name="z_log_var")(encoder_LSTM_latent) # Log variance(s) of the latent space. Log is used to ensure so its exponent (which we'll calculate later) is always positive.

        # A Lambda layer is used to sample from the latent space by passing the mean and log variance to the vae_sampling function
        z = Lambda(Sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])

        encoder = Model(inputs=[x, self.condition_input], outputs=[z_mean, z_log_var, z], name="encoder") # z_mean and z_log_sigma are returned for loss calculation, z_encoder_output is the output of the encoder and will be used as input to the decoder

        # SIDENOTE: Keras is able to trace back the computation graph from the output of the encoder to the input, so it will infer the structure of the encoder from its output and input layers.

        return encoder

    def build_decoder(self):
        """
        Builds the decoder part of the VAE.
        """
        decoder_input = Input(shape=(self.latent_dim,)) # Input to the decoder is the latent space vector z
        condition_input_repeated = Flatten()(RepeatVector(self.latent_dim)(self.condition_input))
        decoder_input_concat = Concatenate()([decoder_input, condition_input_repeated])
        decoder_repeated = RepeatVector(self.time_steps)(decoder_input_concat) # Repeats the latent space vector (z) for the number of time steps, to match the input shape of the LSTM.
        decoder_LSTM_intermediate = RNN(PeepholeLSTMCell(self.int_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True)(decoder_repeated) # Transforms (batch_size, time_steps, latent_dim) to (batch_size, time_steps, int_dim). 
        self.layer_seed += 1
        decoder_LSTM = RNN(PeepholeLSTMCell(self.num_features, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True)(decoder_LSTM_intermediate)
        decoder_output = TimeDistributed(Dense(self.num_features))(decoder_LSTM) # Contains Dense layer at the end to be able to produce high absolute values, since LSTM activations are tanh.
        decoder = Model(inputs=[decoder_input, self.condition_input], outputs=decoder_output, name="decoder") 
        
        return decoder

    @property
    def metrics(self):
            """
            Returns a list of metrics used for tracking during training.
            Returns:list: A list of metrics including total loss, reconstruction loss, and KL divergence loss.
            """
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

    def call(self, inputs):
        """
        Performs the forward pass of the model.
        Args: inputs: The input data.
        Returns: The reconstructed output.
        """
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return reconstruction

    def calculate_losses(self, x, condition):
        """
        Calculates the total loss, reconstruction loss, and KL divergence loss for a given input and condition.

        Parameters:
            x (tensor): The input tensor.
            condition (tensor): The condition tensor.

        Returns:
            total_loss (tensor): The total loss.
            reconstruction_loss (tensor): The reconstruction loss.
            kl_loss (tensor): The KL divergence loss.
        """
        z_mean, z_log_var, z = self.encoder([x, condition])
        reconstruction = self.decoder([z, condition])
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.mse(x, reconstruction),
                axis=(0, 1),
            )
        )
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = self.reconstruction_wt * reconstruction_loss + kl_loss
        return total_loss, reconstruction_loss, kl_loss

    @tf.function
    def train_step(self, data):
        """
        Performs a single training step on the given data.

        Args:
            data: A tuple containing the input data `x` and the condition `condition`.

        Returns:
            A dictionary containing the following metrics:
            - "loss": The total loss value.
            - "reconstruction_loss": The reconstruction loss value.
            - "kl_loss": The KL divergence loss value.
        """
        
        x, condition = data
        with tf.GradientTape() as tape:
            total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    @tf.function
    def test_step(self, data):
        """
        Perform a single testing step of the model.

        Args:
            data (tuple): A tuple containing the input data `x` and the condition `condition`.

        Returns:
            dict: A dictionary containing the calculated losses for the testing step, including the total loss, reconstruction loss, and KL loss.
        """
        x, condition = data
        
        total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def save_weights(self, filepath):
        model_dir, filename = os.path.split(filepath)

        encoder_wts = self.encoder.get_weights()
        decoder_wts = self.decoder.get_weights()
        joblib.dump(encoder_wts, os.path.join(model_dir, f"{filename}_encoder_wts.h5"))
        joblib.dump(decoder_wts, os.path.join(model_dir, f"{filename}_decoder_wts.h5"))

    def load_weights(self, filepath):
        model_dir, filename = os.path.split(filepath)

        print(f"{file_pref}_encoder_wts.h5")
        encoder_wts = joblib.load(os.path.join(model_dir, f"{filename}_encoder_wts.h5"))
        decoder_wts = joblib.load(os.path.join(model_dir, f"{filename}_decoder_wts.h5"))
        self.encoder.set_weights(encoder_wts)
        self.decoder.set_weights(decoder_wts)

    def save(self, filepath, overwrite=True, options=None):
        model_dir, filename = os.path.split(filepath)
        
        params_file = os.path.join(model_dir, f"{filename}_parameters.pkl")

        if not overwrite and os.path.exists(weights_file) and os.path.exists(params_file):
            return

        self.save_weights(filepath)
        
        dict_params = {
            "int_dim": self.int_dim,
            "latent_dim": self.latent_dim,
            "num_features": self.num_features,
            "time_steps": self.time_steps,
            "condition_dim": self.condition_dim,
            "learning_rate": self.learning_rate,
            "batch_size": self.batch_size,
        }
        joblib.dump(dict_params, params_file)
        
    def generate_samples(self, n, condition):
        '''
            Generate random samples from the LSTM VAE.

            n : int : The number of samples to generate.
            condition : numpy array : The condition to generate the samples for.

            Returns
            -------
            A numpy array of shape (n, time_steps, number_of_features) containing the generated samples.
        '''

        # Sample from the standard normal distribution
        z_samples = np.random.normal(size=(n, self.latent_dim))

        # Decode the samples
        gen = self.decoder.predict([z_samples, np.repeat(condition, n, axis=0)])

        return gen
        
class PrintModelPerEpoch(tf.keras.callbacks.Callback):
    def __init__(self, print_weights=False):
        super().__init__()
        self.print_weights = print_weights

    def on_epoch_begin(self, epoch, logs=None):
        print("##########################################")
        for model in [self.model.encoder, self.model.decoder]:
            print(f"Model: {model.name}")
            for layer in model.layers:
                print(f"{layer.name}, input shape: {layer.input_shape}")
                if self.print_weights and layer.weights:
                    for weight in layer.weights:
                        weight_values = weight.numpy()
                        print(f"{layer.name}, weights: {weight.name}, mean: {weight_values.mean()}, variance: {weight_values.var()}")
            if model == self.model.encoder: print("------------------------------------------")
            else: print("##########################################")

In [ ]:
from keras.callbacks import EarlyStopping # , ModelCheckpoint
import pickle 
import os 

# https://discuss.tensorflow.org/t/valueerror-when-saving-autoencoder-tf-example/18618

# Defining hyperparameters
params = {'batch_size': 64, 'int_dim': 36, 'latent_dim': 12, 'learning_rate' : 0.0001}

# Initializing and compiling the model
vae = LSTM_VAE(input_shape=X_train.shape, int_dim=int(params['int_dim']), latent_dim=int(params['latent_dim']), seed=42)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']))

# Define the checkpoint callback
base_model_name = f"model.bs{params['batch_size']}.id{params['int_dim']}.ld{params['latent_dim']}.lr{params['learning_rate']}"

# Add to display inputs and weights per epoch
print_input_shape_callback = PrintModelPerEpoch(print_weights=True)

os.makedirs(f"output/models/{base_model_name}", exist_ok=True)

checkpoint_filepath = f"output/models/{base_model_name}/{base_model_name}"
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', mode="min", patience=10)

# Model fitting
history = vae.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=int(params['batch_size']), callbacks=[early_stopping, checkpoint], epochs=250)

# Save history and best_params as pickle files
with open(f"output/models/{base_model_name}/{base_model_name}_history.pkl", 'wb') as f:
    pickle.dump(history.history, f)

# The model (that are considered the best) can be loaded as -
# vae.load_weights(f"output/ckpt/{base_model_name}")

# print(vae)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate, Flatten, RNN
from tensorflow.keras import backend as K
from tensorflow import shape
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow_addons.rnn import PeepholeLSTMCell
from tensorflow.keras.initializers import GlorotUniform
import os
import joblib

# Source: https://github.com/meliksahturker/LSTM-VAE/tree/main?tab=readme-ov-file 

    class LSTM_VAE(Model):
        def __init__(self, input_shape, int_dim=36, latent_dim=18, condition_dim=1, batch_size=32, learning_rate=0.001, reconstruction_wt = 2, seed=None, **kwargs):
            super().__init__(**kwargs)
            self.int_dim = int_dim
            self.latent_dim = latent_dim
            self.num_features = input_shape[2]
            self.time_steps = input_shape[1]
            self.condition_dim = condition_dim
            self.condition_input = Input(shape=(condition_dim,)) # Shape of the condition input
            self.batch_size = batch_size
            self.learning_rate = learning_rate
            self.reconstruction_wt = reconstruction_wt
            self.seed = seed
            self.layer_seed = 0 
            if self.seed is not None:
                np.random.seed(self.seed)
                tf.random.set_seed(self.seed)
                
            # Everything needed for the encoder and decoder should be defined before calling the encoder and decoder functions
            self.encoder = self.build_encoder()
            self.decoder = self.build_decoder()
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        
        def build_encoder(self):
            """
            Builds the encoder part of the VAE.
            """
    
            def Sampling(args):
                """
                Sampling function for the VAE.
                Parameters: args (tuple): Tuple containing the mean and log variance of the latent space.
                Returns: Sampled latent space vector.
                """
                z_mean, z_log_var = args
                batch_size = shape(z_mean)[0] # Number of samples in the batch
                latent_dim = shape(z_mean)[1] # Dimensionality of the latent space
                epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0, stddev=1) # epsilon = irreducible error. Generates random noise to add to our reparameterisation trick value
                # Reparameterisation trick
                # - `K.exp()` takes the exponential of our log variance to obtain the variance. 
                # - `z_log_var / 2` is equivalent to taking the square root of the variance (standard deviation)
                # - `* epsilon` to get a random value from a normal distribution with mean 0 and standard deviation 1
                return z_mean + K.exp(z_log_var / 2) * epsilon
    
            # --------------------- ENCODER ---------------------
            # Creating the input layer
            x = Input(shape=(self.time_steps, self.num_features))  # Keras adds None to the shape for the batch size: (None, time_steps, number_of_features)
            encoder_input = Concatenate()([x, RepeatVector(self.time_steps)(self.condition_input)]) # Repeat condition for each timsestep. 
    
            # Creating the LSTM layers
            encoder_LSTM_intermediate = RNN(PeepholeLSTMCell(self.int_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True, name="encoder_LSTM_intermediate")(encoder_input)
            self.layer_seed += 1
            encoder_LSTM_latent = RNN(PeepholeLSTMCell(self.latent_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=False, name="encoder_LSTM_latent")(encoder_LSTM_intermediate)
            self.layer_seed += 1
    
            # NOTE: GlorotUniform is equivalent to Xavier activation function (recommended with tanh activation functions, the default in LSTMs).
    
            # These layers' outputs will be trained to represent the mean and log variance of the latent space
            z_mean = Dense(self.latent_dim, name="z_mean")(encoder_LSTM_latent) # Mean(s) of the latent space
            z_log_var = Dense(self.latent_dim, name="z_log_var")(encoder_LSTM_latent) # Log variance(s) of the latent space. Log is used to ensure so its exponent (which we'll calculate later) is always positive.
    
            # A Lambda layer is used to sample from the latent space by passing the mean and log variance to the vae_sampling function
            z = Lambda(Sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])
    
            encoder = Model(inputs=[x, self.condition_input], outputs=[z_mean, z_log_var, z], name="encoder") # z_mean and z_log_sigma are returned for loss calculation, z_encoder_output is the output of the encoder and will be used as input to the decoder
    
            # SIDENOTE: Keras is able to trace back the computation graph from the output of the encoder to the input, so it will infer the structure of the encoder from its output and input layers.
    
            return encoder
    
        def build_decoder(self):
            """
            Builds the decoder part of the VAE.
            """
            decoder_input = Input(shape=(self.latent_dim,)) # Input to the decoder is the latent space vector z
            condition_input_repeated = Flatten()(RepeatVector(self.latent_dim)(self.condition_input))
            decoder_input_concat = Concatenate()([decoder_input, condition_input_repeated])
            decoder_repeated = RepeatVector(self.time_steps)(decoder_input_concat) # Repeats the latent space vector (z) for the number of time steps, to match the input shape of the LSTM.
            decoder_LSTM_intermediate = RNN(PeepholeLSTMCell(self.int_dim, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True)(decoder_repeated) # Transforms (batch_size, time_steps, latent_dim) to (batch_size, time_steps, int_dim). 
            self.layer_seed += 1
            decoder_LSTM = RNN(PeepholeLSTMCell(self.num_features, kernel_initializer=GlorotUniform(seed=self.seed + self.layer_seed)), return_sequences=True)(decoder_LSTM_intermediate)
            decoder_output = TimeDistributed(Dense(self.num_features))(decoder_LSTM) # Contains Dense layer at the end to be able to produce high absolute values, since LSTM activations are tanh.
            decoder = Model(inputs=[decoder_input, self.condition_input], outputs=decoder_output, name="decoder") 
            
            return decoder
    
        @property
        def metrics(self):
                """
                Returns a list of metrics used for tracking during training.
                Returns:list: A list of metrics including total loss, reconstruction loss, and KL divergence loss.
                """
                return [
                    self.total_loss_tracker,
                    self.reconstruction_loss_tracker,
                    self.kl_loss_tracker,
                ]
    
        def call(self, inputs):
            """
            Performs the forward pass of the model.
            Args: inputs: The input data.
            Returns: The reconstructed output.
            """
            z_mean, z_log_var, z = self.encoder(inputs)
            reconstruction = self.decoder(z)
            return reconstruction
    
        def calculate_losses(self, x, condition):
            """
            Calculates the total loss, reconstruction loss, and KL divergence loss for a given input and condition.
    
            Parameters:
                x (tensor): The input tensor.
                condition (tensor): The condition tensor.
    
            Returns:
                total_loss (tensor): The total loss.
                reconstruction_loss (tensor): The reconstruction loss.
                kl_loss (tensor): The KL divergence loss.
            """
            z_mean, z_log_var, z = self.encoder([x, condition])
            reconstruction = self.decoder([z, condition])
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(x, reconstruction),
                    axis=(0, 1),
                )
            )
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            reconstruction_loss = self.reconstruction_wt * reconstruction_loss
            total_loss = reconstruction_loss + kl_loss
            return total_loss, reconstruction_loss, kl_loss
    
        @tf.function
        def train_step(self, data):
            """
            Performs a single training step on the given data.
    
            Args:
                data: A tuple containing the input data `x` and the condition `condition`.
    
            Returns:
                A dictionary containing the following metrics:
                - "loss": The total loss value.
                - "reconstruction_loss": The reconstruction loss value.
                - "kl_loss": The KL divergence loss value.
            """
            
            x, condition = data
            with tf.GradientTape() as tape:
                total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
                
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }
    
        @tf.function
        def test_step(self, data):
            """
            Perform a single testing step of the model.
    
            Args:
                data (tuple): A tuple containing the input data `x` and the condition `condition`.
    
            Returns:
                dict: A dictionary containing the calculated losses for the testing step, including the total loss, reconstruction loss, and KL loss.
            """
            x, condition = data
            
            total_loss, reconstruction_loss, kl_loss = self.calculate_losses(x, condition)
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }
    
        def save_weights(self, filepath):
            model_dir, filename = os.path.split(filepath)
    
            encoder_wts = self.encoder.get_weights()
            decoder_wts = self.decoder.get_weights()
            joblib.dump(encoder_wts, os.path.join(model_dir, f"{filename}_encoder_wts.h5"))
            joblib.dump(decoder_wts, os.path.join(model_dir, f"{filename}_decoder_wts.h5"))
    
        def load_weights(self, filepath):
            model_dir, filename = os.path.split(filepath)
    
            print(f"{file_pref}_encoder_wts.h5")
            encoder_wts = joblib.load(os.path.join(model_dir, f"{filename}_encoder_wts.h5"))
            decoder_wts = joblib.load(os.path.join(model_dir, f"{filename}_decoder_wts.h5"))
            self.encoder.set_weights(encoder_wts)
            self.decoder.set_weights(decoder_wts)
    
        def save(self, filepath, overwrite=True, options=None):
            model_dir, filename = os.path.split(filepath)
            
            params_file = os.path.join(model_dir, f"{filename}_parameters.pkl")
    
            if not overwrite and os.path.exists(weights_file) and os.path.exists(params_file):
                return
    
            self.save_weights(filepath)
            
            dict_params = {
                "int_dim": self.int_dim,
                "latent_dim": self.latent_dim,
                "num_features": self.num_features,
                "time_steps": self.time_steps,
                "condition_dim": self.condition_dim,
                "learning_rate": self.learning_rate,
                "batch_size": self.batch_size,
            }
            joblib.dump(dict_params, params_file)
            
        def generate_samples(self, n, condition):
            '''
                Generate random samples from the LSTM VAE.
    
                n : int : The number of samples to generate.
                condition : numpy array : The condition to generate the samples for.
    
                Returns
                -------
                A numpy array of shape (n, time_steps, number_of_features) containing the generated samples.
            '''
    
            # Sample from the standard normal distribution
            z_samples = np.random.normal(size=(n, self.latent_dim))
    
            # Decode the samples
            gen = self.decoder.predict([z_samples, np.repeat(condition, n, axis=0)])
    
            return gen
            
    class PrintModelPerEpoch(tf.keras.callbacks.Callback):
        def __init__(self, print_weights=False):
            super().__init__()
            self.print_weights = print_weights
    
        def on_epoch_begin(self, epoch, logs=None):
            print("##########################################")
            for model in [self.model.encoder, self.model.decoder]:
                print(f"Model: {model.name}")
                for layer in model.layers:
                    print(f"{layer.name}, input shape: {layer.input_shape}")
                    if self.print_weights and layer.weights:
                        for weight in layer.weights:
                            weight_values = weight.numpy()
                            print(f"{layer.name}, weights: {weight.name}, mean: {weight_values.mean()}, variance: {weight_values.var()}")
                if model == self.model.encoder: print("------------------------------------------")
                else: print("##########################################")
except Exception as e:
    print(f"An error occurred: {e}")
else:
    print("No errors occurred. LSTM-VAE class was successfully loaded.")

## `optimise_LSTM_VAE.py`

In [ ]:
# %pip install tensorflow pandas numpy matplotlib seaborn numpy scikit-learn hyperopt tensorflow_addons pydot graphviz visualkeras reload imblearn neurokit2

import helpers as h
from importlib import reload
from LSTM_VAE import LSTM_VAE
import numpy as np
from tensorflow import keras

reload(h)

try:
    X_train, X_val, X_test, y_train, y_val, y_test, p_train, p_val, p_test = h.prepare_train_val_test_sets(filenames=['input/dl_X_wl24_sr32.pkl', 'input/dl_y_wl24_sr32.pkl', 'input/dl_p_wl24_sr32.pkl'])
except Exception as e:
    print(f"An error occurred: {e}")
else:
    print("No errors occurred. Data partitioned successfully.")

#--------------------- Hyperparameter optimisation ---------------------------# 
from hyperopt import hp
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from hyperopt import fmin, tpe, Trials
import pickle

filename = "240430_hyperopt_best_params"

try:
    # Define the hyperparameter space
    space = {
        'batch_size': hp.choice('batch_size', [32, 64]),
        'int_dim': hp.choice('int_dim', [25, 50, 75, 100, 125, 150, 175, 200]),
        'latent_dim': hp.choice('latent_dim', [7, 8, 10, 12, 14, 18, 24, 32, 48, 72, 96, 120]),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
        'reconstruction_wt': hp.choice('reconstruction_wt', [1, 2, 3]),
        'optimizer': hp.choice('optimizer', ['Adam', 'RMSprop'])
    }

    # Objective function
    def objective(params):
        vae = LSTM_VAE(input_shape=X_train.shape, int_dim=int(params['int_dim']), latent_dim=int(params['latent_dim']), reconstruction_wt = int(params['reconstruction_wt']), seed=42)    

        if params['optimizer'] == 'Adam':
            opt = keras.optimizers.Adam(learning_rate=params['learning_rate'])
        elif params['optimizer'] == 'RMSprop':
            opt = keras.optimizers.RMSprop(learning_rate=params['learning_rate'])

        vae.compile(optimizer=opt)

        early_stopping = EarlyStopping(monitor='val_loss', mode="min", patience=10)
        history = vae.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=int(params['batch_size']), callbacks=[early_stopping], epochs=1000, verbose=2) # callbacks=[early_stopping, checkpoint]
        return np.min(history.history['val_loss'])

    best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=Trials())

    with open(f"{filename}.pkl", 'wb') as f:
        pickle.dump(best_params, f)
except Exception as e:
    print(f"An error occurred: {e}")
else: 
    print(f"No errors occurred. Hyperparameter optimisation procedure completed successfully. Best parameters stored as: {filename}.pkl")

#--------------------- Training and storing the model ---------------------------# 
import os # https://discuss.tensorflow.org/t/valueerror-when-saving-autoencoder-tf-example/18618

# Defining hyperparameters
print("BEST PARAMETERS:\n", best_params)
params = best_params

# Initializing and compiling the model
vae = LSTM_VAE(input_shape=X_train.shape, int_dim=int(params['int_dim']), latent_dim=int(params['latent_dim']), reconstruction_wt = int(params['reconstruction_wt']), seed=42)    
if params['optimizer'] == 'Adam':
    opt = keras.optimizers.Adam(learning_rate=params['learning_rate'])
elif params['optimizer'] == 'RMSprop':
    opt = keras.optimizers.RMSprop(learning_rate=params['learning_rate'])
vae.compile(optimizer=opt)

# Define the checkpoint callback
base_model_name = f"model.bs{params['batch_size']}.id{params['int_dim']}.ld{params['latent_dim']}.lr{params['learning_rate']}.rw{params['reconstruction_wt']}"
os.makedirs(f"output/models/{base_model_name}", exist_ok=True)

checkpoint_filepath = f"output/models/{base_model_name}/{base_model_name}"
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Defining Early stopping
early_stopping = EarlyStopping(monitor='val_loss', mode="min", patience=10, restore_best_weights = True)

# Fitting the model
history = vae.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=int(params['batch_size']), callbacks=[early_stopping, checkpoint], epochs=1000, verbose=2)

# Save history and best_params as pickle files
with open(f"output/models/{base_model_name}/{base_model_name}_history.pkl", 'wb') as f:
    pickle.dump(history.history, f)